In [104]:
import pandas as pd
import glob
import os
import geopandas 
import geopy 
from geopy.geocoders import Nominatim


In [6]:
data = pd.concat([pd.read_excel(f, 'Results') for f in glob.glob('/Users/gocchini/Desktop/lit_paper_3/data/*.xlsx')], ignore_index = True)

In [8]:
data.to_pickle('/Users/gocchini/Desktop/lit_paper_3/data/data_almost_complete.pkl')

In [46]:
london_postcodes = pd.read_csv('/Users/gocchini/Desktop/lit_paper_3/london_postcodes-ons-postcodes-directory-feb22.csv')

In [97]:
def data_processing(data, london_data): 
    data = data.loc[(data['Inactive'] == 'No')]
    data = data[data['Primary UK SIC (2007) code'].notna()]
    data = data[data['Primary trading address'].notna()]
    data = data.drop_duplicates(keep=False)
    data['postcode'] = data['Primary trading address'].apply(lambda x: x.split(',')[-1])
    data['postcode'] = data['postcode'].apply(lambda x: x[1:])
    london_postcodes = london_data['pcd'].tolist()
    london_df = data[data['postcode'].isin(london_postcodes)]
    return london_df 

In [100]:
london_df = data_processing(data , london_postcodes)

In [135]:
def coordinates_finder(address):
    locator = Nominatim(user_agent='myGeocoder')
    location = locator.geocode(address)
    try:
        coordinates = '{}, {}'.format(location.latitude, location.longitude)
        print(coordinates)
    except AttributeError:
        coordinates = 'not found'
    return coordinates

In [ ]:
london_df['coordinates'] = london_df['Primary trading address'].apply(lambda x: coordinates_finder(x))

https://data.london.gov.uk/dataset/postcode-directory-for-london